# Working with xAI Grok in Pixeltable

Pixeltable's xAI integration enables you to use Grok models for chat, reasoning, and image generation via the native xAI SDK.

### Prerequisites
- An xAI account with an API key (https://console.x.ai/)

### Important notes

- xAI API usage incurs costs based on your xAI plan
- The native xAI SDK (`xai-sdk`) provides access to the latest features including reasoning models
- The OpenAI-compatible endpoint is also supported for backward compatibility
- Be mindful of sensitive data and consider security measures when integrating with external services


First you'll need to install the xAI SDK and set up your xAI API key.

```bash
pip install xai-sdk
```


In [ ]:
# For local development, use the local pixeltable path
import sys
sys.path.insert(0, '/Users/pierre/pixeltable')


In [ ]:
import os
import getpass

if 'XAI_API_KEY' not in os.environ:
    os.environ['XAI_API_KEY'] = getpass.getpass('Enter your xAI API key: ')


Now let's create a Pixeltable directory to hold the tables for our demo.


In [ ]:
import pixeltable as pxt

# Remove the 'xai_demo' directory and its contents, if it exists
pxt.drop_dir('xai_demo', force=True)
pxt.create_dir('xai_demo')


## Chat with Grok

Grok is xAI's flagship language model, designed to provide truthful, insightful answers. The `xai.chat` UDF uses the native xAI SDK which supports the latest features including the Responses API.


from pixeltable.functions import xai

# Create a table for chat conversations
chat_t = pxt.create_table('xai_demo.chat', {'prompt': pxt.String})

# Create message format for Grok
messages = [
    {'role': 'system', 'content': 'You are Grok, a helpful AI assistant created by xAI.'},
    {'role': 'user', 'content': chat_t.prompt}
]

# Add computed column using the native xAI chat UDF
chat_t.add_computed_column(
    response=xai.chat(messages=messages, model='grok-4')
)

# Extract just the response text
chat_t.add_computed_column(
    answer=chat_t.response['content']
)


In [ ]:
# Insert some prompts
chat_t.insert([
    {'prompt': 'What is the meaning of life, the universe, and everything?'},
    {'prompt': 'Explain quantum computing in simple terms.'},
    {'prompt': 'What makes a good cup of coffee?'},
])


In [ ]:
# View the results
chat_t.select(chat_t.prompt, chat_t.answer).collect()


## Reasoning Models

The `grok-3-mini` model supports extended reasoning with the `reasoning_effort` parameter. This allows the model to "think" longer for complex problems.


In [ ]:
# Create a table for reasoning tasks
reasoning_t = pxt.create_table('xai_demo.reasoning', {'problem': pxt.String})

# Create message format
messages = [
    {'role': 'system', 'content': 'You are a math expert. Show your work step by step.'},
    {'role': 'user', 'content': reasoning_t.problem}
]

# Use reasoning model with high effort for complex problems
reasoning_t.add_computed_column(
    response=xai.chat(
        messages=messages,
        model='grok-3-mini',
        reasoning_effort='high'
    )
)

# Extract answer and reasoning tokens used
reasoning_t.add_computed_column(answer=reasoning_t.response['content'])
reasoning_t.add_computed_column(reasoning_tokens=reasoning_t.response['usage']['reasoning_tokens'])


In [ ]:
# Insert math problems
reasoning_t.insert([
    {'problem': 'What is 101 * 3?'},
    {'problem': 'If a train travels 120 miles in 2 hours, what is its average speed?'},
])


In [ ]:
# View the results with reasoning tokens
reasoning_t.select(reasoning_t.problem, reasoning_t.answer, reasoning_t.reasoning_tokens).collect()


## Image Generation

xAI also offers image generation capabilities with Grok models.


# Create a table for image generation
img_t = pxt.create_table('xai_demo.images', {'prompt': pxt.String})

# Add computed column for image generation
img_t.add_computed_column(
    generated_image=xai.image_generations(img_t.prompt, model='grok-2-image')
)


In [ ]:
# Insert image prompts
img_t.insert([
    {'prompt': 'A futuristic cityscape at sunset with flying cars'},
    {'prompt': 'A cute robot helping in a garden'},
])


In [ ]:
# View the generated images
img_t.select(img_t.prompt, img_t.generated_image).collect()


## Available Models

### Language Models

| Model | Description |
|-------|-------------|
| `grok-4` | Latest and most capable Grok 4 model |
| `grok-4-fast` | Faster Grok 4 variant |
| `grok-3` | Grok 3 model |
| `grok-3-fast` | Faster Grok 3 variant |
| `grok-3-mini` | Reasoning model with `reasoning_effort` support |
| `grok-2-vision-1212` | Grok 2 with vision capabilities |
| `grok-code-fast-1` | Code-optimized model |

### Image Generation Models

| Model | Description |
|-------|-------------|
| `grok-2-image` | Image generation model |

### Chat Parameters

The `chat` function supports:

- `model`: The Grok model to use
- `reasoning_effort`: `'low'` or `'high'` (only for `grok-3-mini`)
- `max_tokens`: Maximum tokens in the response
- `temperature`: Sampling temperature (0-2)
- `top_p`: Nucleus sampling parameter
- `store_messages`: Enable stateful Responses API


## Learn More

- [xAI Documentation](https://docs.x.ai/)
- [xAI Console](https://console.x.ai/)
- [Grok Models](https://docs.x.ai/docs/models)
- [Responses API Guide](https://docs.x.ai/docs/guides/responses-api)
- [Reasoning Models Guide](https://docs.x.ai/docs/guides/reasoning)
